In [1]:
import os
import json
import argparse
import numpy as np
import torch
from datasets import load_dataset
from transformers import GenerationConfig, AutoConfig, AutoTokenizer, BitsAndBytesConfig
from vllm import LLM, SamplingParams
import re
import math
from math_verify import parse, verify, LatexExtractionConfig
from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 08-29 03:45:50 [__init__.py:239] Automatically detected platform cuda.


2025-08-29 03:45:52,728	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
DATASET_MAP = {
    "MATH-500": {
        "args": ("HuggingFaceH4/MATH-500", "test"),
        "question_key": "problem",
        "answer_key": "answer"
    },
    "AIME2024": {
        "args": ("HuggingFaceH4/aime_2024", "train"),
        "question_key": "problem",
        "answer_key": "answer"
    },
    "gpqa": {
        "args": ("hendrydong/gpqa_diamond_mc", "test"),
        "question_key": "problem",
        "answer_key": "solution"
    },
    "gsm8k": {
        "args": ("skrishna/gsm8k_only_answer", "test"),
        "question_key": "text",
        "answer_key": "label"
    },
    "openr1-math": {
        "args": ("open-r1/OpenR1-Math-220k", "train"),
        "question_key": "problem",
        "answer_key": "answer"
    },
    "AIME2025": {
        "args": ("yentinglin/aime_2025", "train"),
        "question_key": "problem",
        "answer_key": "answer"
    },
    "MMLU-Pro-math": {
        "args": ("TIGER-Lab/MMLU-Pro", "test"),
        "options_key": "options",
        "question_key": "question",
        "answer_key": "answer"
    }
}

In [3]:
MODEL_MAP   = {
    "deepseek-qwen-1.5b": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "deepseek-llama3-8b": "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "deepseek-qwen-14b": "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B",
    "qwq-32b": "Qwen/QwQ-32B",
    "qwen3-8b": "Qwen/Qwen3-8B",
    "deepseek-qwen3-8b": "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
    "phi4-reasoning-plus": "microsoft/Phi-4-reasoning-plus",
    "nemotron-7b": "nvidia/OpenMath-Nemotron-7B",
}

In [5]:
dataset = "gsm8k"
model = "deepseek-llama3-8b"

In [6]:
dataset_name, split = DATASET_MAP[dataset]["args"]
ds = load_dataset(dataset_name, split=split)
question_key = DATASET_MAP[dataset]["question_key"]
answer_key   = DATASET_MAP[dataset]["answer_key"]

Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 84492.07 examples/s]


In [7]:
model_id = MODEL_MAP[model]
max_pos = AutoConfig.from_pretrained(model_id).max_position_embeddings
cfg = GenerationConfig.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",       
#     bnb_4bit_compute_dtype=torch.float16
# )

llm = LLM(
    model=model_id,
    max_model_len=4096,
    dtype="half",
    # gpu_memory_utilization=0.7,
    # quantization="bitsandbytes"
)

WARNING 08-27 07:02:47 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 08-27 07:03:02 [config.py:717] This model supports multiple tasks: {'score', 'embed', 'classify', 'reward', 'generate'}. Defaulting to 'generate'.
WARNING 08-27 07:03:02 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 08-27 07:03:02 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5) with config: model='deepseek-ai/DeepSeek-R1-Distill-Llama-8B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Llama-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfi

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:07<00:07,  7.04s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:14<00:00,  7.01s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:14<00:00,  7.02s/it]



INFO 08-27 07:05:43 [loader.py:458] Loading weights took 14.27 seconds
INFO 08-27 07:05:43 [model_runner.py:1140] Model loading took 14.9889 GiB and 154.309636 seconds
INFO 08-27 07:05:45 [worker.py:287] Memory profiling takes 1.23 seconds
INFO 08-27 07:05:45 [worker.py:287] the current vLLM instance can use total_gpu_memory (31.73GiB) x gpu_memory_utilization (0.90) = 28.56GiB
INFO 08-27 07:05:45 [worker.py:287] model weights take 14.99GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.20GiB; the rest of the memory reserved for KV Cache is 12.28GiB.
INFO 08-27 07:05:45 [executor_base.py:112] # cuda blocks: 6287, # CPU blocks: 2048
INFO 08-27 07:05:45 [executor_base.py:117] Maximum concurrency for 4096 tokens per request: 24.56x
INFO 08-27 07:05:49 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the 

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:37<00:00,  1.06s/it]

INFO 08-27 07:06:26 [model_runner.py:1592] Graph capturing finished in 37 secs, took 0.24 GiB
INFO 08-27 07:06:26 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 42.44 seconds


In [9]:
def make_params(n: int, budget: int, cfg) -> SamplingParams:
    """
    Build SamplingParams from model config and given budget.
    """
    kw = {"n": n, "max_tokens": budget}
    if hasattr(cfg, "temperature") and cfg.temperature is not None:
        kw["temperature"] = cfg.temperature
    if hasattr(cfg, "top_k") and cfg.top_k is not None:
        kw["top_k"] = cfg.top_k
    if hasattr(cfg, "top_p") and cfg.top_p is not None:
        kw["top_p"] = cfg.top_p
    return SamplingParams(**kw)

In [26]:
def apply_chat(prompt: str, tokenizer):
    """
    Wraps a user prompt in the vLLM chat template.
    """
    conversations = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(
        conversations,
        tokenize=False,
        add_generation_prompt=True
    )

In [27]:
prompts = []
for ex in tqdm(ds):
    q = ex[question_key]
    prompt = (
                f"Problem: {q}\n\n"
                "Please reason step by step, and put your final answer within \\boxed{}."
            )
    prompts.append(apply_chat(prompt, tokenizer))

100%|██████████| 1319/1319 [00:00<00:00, 6828.61it/s]


In [33]:
sampling_params = SamplingParams(n=1, temperature=0.0, max_tokens=3072)

In [31]:
llm.generate(prompts[0], sampling_params=sampling_params)

TypeError: LLM.generate() got an unexpected keyword argument 'do_sample'

In [ ]:
results = llm.generate(prompts=prompts, sampling_params=sampling_params)

Processed prompts:   0%|          | 0/6595 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 08-27 00:54:52 [scheduler.py:1768] Sequence group 51_parallel_sample_0 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1
WARNING 08-27 00:55:03 [scheduler.py:1768] Sequence group 41_parallel_sample_0 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51
WARNING 08-27 00:55:21 [scheduler.py:1768] Sequence group 30_parallel_sample_4 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_pree

Processed prompts:   0%|          | 20/6595 [02:09<7:29:46,  4.10s/it, est. speed input: 11.49 toks/s, output: 138.95 toks/s]

WARNING 08-27 00:56:28 [scheduler.py:1768] Sequence group 26_parallel_sample_0 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=151


Processed prompts:   1%|▏         | 95/6595 [04:07<2:18:18,  1.28s/it, est. speed input: 28.91 toks/s, output: 469.42 toks/s]

...

WARNING 08-27 01:27:29 [scheduler.py:1768] Sequence group 251_parallel_sample_4 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=3551


Processed prompts:  17%|█▋        | 1120/6595 [34:04<2:02:30,  1.34s/it, est. speed input: 44.47 toks/s, output: 846.38 toks/s]

WARNING 08-27 01:28:22 [scheduler.py:1768] Sequence group 254_parallel_sample_0 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=3601


Processed prompts:  17%|█▋        | 1135/6595 [34:35<2:31:29,  1.66s/it, est. speed input: 44.57 toks/s, output: 849.52 toks/s]

WARNING 08-27 01:28:51 [scheduler.py:1768] Sequence group 261_parallel_sample_2 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=3651


In [15]:
def verify_answer(pred: str, ref: str) -> bool:

    # ── patterns & threshold ─────────────────────────────────────────────────
    BASE_N_RE    = re.compile(r"^\(?([0-9A-Za-z]+)\)?_\{(\d+)\}$")
    EXP_RE       = re.compile(r"\^\{(\d+)\}")
    MAX_SAFE_EXP = 10_000

    # ── normalize inputs ─────────────────────────────────────────────────────
    if pred is None or ref is None:
        return False
    p = pred.strip()
    r = ref.strip()

    # ── 1) base-N literal in prediction ─────────────────────────────────────
    m = BASE_N_RE.match(p)
    if m:
        return m.group(1) == r

    # ── 2) base-N literal in reference ──────────────────────────────────────
    m = BASE_N_RE.match(r)
    if m:
        return m.group(1) == p

    # ── 3) huge-exponent guard ───────────────────────────────────────────────
    exps = [int(e) for e in EXP_RE.findall(p)]
    if exps and max(exps) > MAX_SAFE_EXP:
        return p.replace(" ", "") == r.replace(" ", "")

    # ── 4) fallback to math_verify ──────────────────────────────────────────
    wrap = lambda s: f"\\({s}\\)"
    cfg  = LatexExtractionConfig()
    try:
        g_node = parse(wrap(r), extraction_config=[cfg])
        p_node = parse(wrap(p), extraction_config=[cfg])
        return verify(g_node, p_node, float_rounding=2)
    except Exception:
        return False

def extract_answer(text):
    if text is None:
        return None
    # Step 1: Remove everything that is not a number, letter, ".", or "-"
    # text = re.sub(r'[^0-9a-zA-Z{}\\.\-]', '', text)
    # Try extracting from 'boxed' first
    boxed_matches = extract_boxed(text)
    if boxed_matches:
        extracted_answer = boxed_matches[-1][1:-1]
        return extracted_answer

    # Fallback: extract any numbers
    numbers = re.findall(r'-?\d+\.\d+|-?\d+', text)
    if not numbers:
        return None

    try:
        extracted_number = float(numbers[-1])
        # Guard against infinity
        if math.isinf(extracted_number):
            return None
        
        return numbers[-1]
    except (ValueError, OverflowError):
        return None

def extract_boxed(text):
    pattern = re.compile(r'boxed\{')
    matches = []
    stack = []
    
    i = 0
    while i < len(text):
        match = pattern.search(text, i)
        if not match:
            break
        
        start = match.end() - 1  # Position at the first `{`
        stack.append(start)
        i = start + 1
        count = 1  # To track `{}` pairs
        
        while i < len(text) and stack:
            if text[i] == '{':
                count += 1
            elif text[i] == '}':
                count -= 1
                if count == 0:  # Found a matching closing `}`
                    start = stack.pop()
                    matches.append(text[start:i+1])
                    break
            i += 1
    
    return matches

In [17]:
runs = {rid: [] for rid in range(10)}
for idx, gen in tqdm(enumerate(results)):
    gold = ds[idx][answer_key]
    for rid, out in enumerate(gen.outputs):
        text = out.text.strip()
        # prediction extraction
        pred = extract_answer(text)
        # correctness
        correct = False
        try:
            correct = verify_answer(gold, pred)
        except:
            pass
        # reasoning length (entire response) in tokens
        reasoning_length = len(tokenizer.encode(text, add_special_tokens=False))

        runs[rid].append({
            "question":         ds[idx][question_key],
            "full_response":    text,
            "reasoning_length": reasoning_length,
            "prediction":       pred,
            "gold":             gold,
            "correct":          correct
        })

1319it [02:13,  9.87it/s]


In [18]:
os.makedirs("hint_results/gsm8k/deepseek-qwen-1.5b", exist_ok=True)
output_path = (
    "hint_results/gsm8k/deepseek-qwen-1.5b/"
    "10_runs.json"
)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump({"runs":[{"run_id":rid,"records":recs} for rid,recs in runs.items()]}, f, indent=4)